In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.device_count()

1

In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


c:\Users\AdMiN\miniconda3\envs\py312-tonk-chatbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


c:\Users\AdMiN\miniconda3\envs\py312-tonk-chatbot\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:330: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"cuda:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    NVIDIA GeForce GTX 1060 6GB. Num GPUs = 1. Max memory: 6.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu126. CUDA: 6.1. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [6]:
from datasets import load_dataset
dataset = load_dataset(
    "csv",
    data_files = "https://huggingface.co/datasets/unsloth/datasets/raw/main/titanic.csv",
    split = "train",
)
print(dataset.column_names)
print(dataset[0])

['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
{'PassengerId': 1, 'Survived': 0, 'Pclass': 3, 'Name': 'Braund, Mr. Owen Harris', 'Sex': 'male', 'Age': 22.0, 'SibSp': 1, 'Parch': 0, 'Ticket': 'A/5 21171', 'Fare': 7.25, 'Cabin': None, 'Embarked': 'S'}


In [7]:
from unsloth import to_sharegpt
dataset_simple = to_sharegpt(
    dataset,
    merged_prompt = "[[Their age is {Age}.\n]][[They paid ${Fare} for the trip.\n]]",
    output_column_name = "Survived",
)

In [10]:
from unsloth import to_sharegpt
dataset = to_sharegpt(
    dataset,
    merged_prompt = \
        "[[The passenger embarked from {Embarked}.]]"\
        "[[\nThey are {Sex}.]]"\
        "[[\nThey have {Parch} parents and childen.]]"\
        "[[\nThey have {SibSp} siblings and spouses.]]"\
        "[[\nTheir passenger class is {Pclass}.]]"\
        "[[\nTheir age is {Age}.]]"\
        "[[\nThey paid ${Fare} for the trip.]]",
    conversation_extension = 5, # Randomnly combines conversations into 1! Good for long convos
    output_column_name = "Survived",
)

In [11]:
dataset[0]

{'conversations': [{'from': 'human',
   'value': 'Their age is 22.0.\nThey paid $7.25 for the trip.\n'},
  {'from': 'gpt', 'value': '0'},
  {'from': 'human',
   'value': 'Their age is 52.0.\nThey paid $79.65 for the trip.\n'},
  {'from': 'gpt', 'value': '0'},
  {'from': 'human',
   'value': 'Their age is 9.0.\nThey paid $31.275 for the trip.\n'},
  {'from': 'gpt', 'value': '0'},
  {'from': 'human', 'value': 'They paid $7.8958 for the trip.\n'},
  {'from': 'gpt', 'value': '0'},
  {'from': 'human',
   'value': 'Their age is 24.0.\nThey paid $13.0 for the trip.\n'},
  {'from': 'gpt', 'value': '0'}]}

In [12]:
from unsloth import standardize_sharegpt
dataset = standardize_sharegpt(dataset)

In [13]:
dataset[0]

{'conversations': [{'content': 'Their age is 22.0.\nThey paid $7.25 for the trip.\n',
   'role': 'user'},
  {'content': '0', 'role': 'assistant'},
  {'content': 'Their age is 52.0.\nThey paid $79.65 for the trip.\n',
   'role': 'user'},
  {'content': '0', 'role': 'assistant'},
  {'content': 'Their age is 9.0.\nThey paid $31.275 for the trip.\n',
   'role': 'user'},
  {'content': '0', 'role': 'assistant'},
  {'content': 'They paid $7.8958 for the trip.\n', 'role': 'user'},
  {'content': '0', 'role': 'assistant'},
  {'content': 'Their age is 24.0.\nThey paid $13.0 for the trip.\n',
   'role': 'user'},
  {'content': '0', 'role': 'assistant'}]}

In [14]:
chat_template = """Below describes some details about some passengers who went on the Titanic.
Predict whether they survived or perished based on their characteristics.
Output 1 if they survived, and 0 if they died.
>>> Passenger Details:
{INPUT}
>>> Did they survive?
{OUTPUT}"""

from unsloth import apply_chat_template
dataset = apply_chat_template(
    dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
    # default_system_message = "You are a helpful assistant", << [OPTIONAL]
)

Unsloth: We automatically added an EOS token to stop endless generations.
Map: 100%|██████████| 891/891 [00:00<00:00, 6345.49 examples/s]


In [15]:
dataset[0]

{'conversations': [{'content': 'Their age is 22.0.\nThey paid $7.25 for the trip.\n',
   'role': 'user'},
  {'content': '0', 'role': 'assistant'},
  {'content': 'Their age is 52.0.\nThey paid $79.65 for the trip.\n',
   'role': 'user'},
  {'content': '0', 'role': 'assistant'},
  {'content': 'Their age is 9.0.\nThey paid $31.275 for the trip.\n',
   'role': 'user'},
  {'content': '0', 'role': 'assistant'},
  {'content': 'They paid $7.8958 for the trip.\n', 'role': 'user'},
  {'content': '0', 'role': 'assistant'},
  {'content': 'Their age is 24.0.\nThey paid $13.0 for the trip.\n',
   'role': 'user'},
  {'content': '0', 'role': 'assistant'}],
 'text': '<|begin_of_text|>Below describes some details about some passengers who went on the Titanic.\nPredict whether they survived or perished based on their characteristics.\nOutput 1 if they survived, and 0 if they died.\n>>> Passenger Details:\nTheir age is 22.0.\nThey paid $7.25 for the trip.\n\n>>> Did they survive?\n0<|end_of_text|>\n>>> Pa

In [16]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/891 [00:02<?, ? examples/s]


RuntimeError: One of the subprocesses has abruptly died during map operation.To debug the error, disable multiprocessing.

In [105]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files='./S05_SelectGoodScore/ALLALL.csv', split='train')

In [106]:
from unsloth import standardize_sharegpt
dataset = standardize_sharegpt(dataset)

In [107]:
dataset[0]

{'INPUT': "What are the main components of an industrial gas turbine as mentioned in the book 'Industrial Gas Turbines: Performance and Operability' by A. M. Y Razak?",
 'OUTPUT': 'Heat exchanger, combustor, compressor, gas supply, and heat sinks are highlighted as essential parts in the book.'}

In [108]:
chat_template = """Answer question from the manual.
>>> User Question:
{INPUT}
>>> Assistant Answer?
{OUTPUT}"""

from unsloth import apply_chat_template
dataset = apply_chat_template(
    dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
    # default_system_message = "You are a helpful assistant", << [OPTIONAL]
)

Unsloth: We automatically added an EOS token to stop endless generations.
Map:   0%|          | 0/8172 [00:00<?, ? examples/s]


KeyError: 'conversations'